# Data Science Interview Simulation: Practical Scenarios
**Duration: ~60 minutes**

This notebook simulates a technical interview environment with realistic data scenarios. You'll work through progressively challenging problems that test your SQL, data modeling, Python, and analytical skills.

## Instructions
- Read each scenario carefully
- Write your solution in the provided code cells
- Test your code to ensure it works
- Be prepared to explain your approach

---

## Setup: Museum Database

You're interviewing for a data analyst position at a museum that tracks visitors, exhibitions, and artwork. First, let's set up the database.

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta

# Create database connection
conn = sqlite3.connect('museum_interview.db')
cursor = conn.cursor()

print("Database connection established")

Database connection established


In [2]:
# Create the museum database schema

# Artists table
cursor.execute('''
CREATE TABLE IF NOT EXISTS artists (
    artist_id INTEGER PRIMARY KEY,
    artist_name TEXT NOT NULL,
    birth_year INTEGER,
    nationality TEXT,
    movement TEXT
)
''')

# Artworks table
cursor.execute('''
CREATE TABLE IF NOT EXISTS artworks (
    artwork_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    artist_id INTEGER,
    creation_year INTEGER,
    medium TEXT,
    value_usd DECIMAL(12, 2),
    FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
)
''')

# Exhibitions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS exhibitions (
    exhibition_id INTEGER PRIMARY KEY,
    exhibition_name TEXT NOT NULL,
    start_date DATE,
    end_date DATE,
    gallery TEXT,
    ticket_price DECIMAL(6, 2)
)
''')

# Exhibition_Artworks (many-to-many relationship)
cursor.execute('''
CREATE TABLE IF NOT EXISTS exhibition_artworks (
    exhibition_id INTEGER,
    artwork_id INTEGER,
    PRIMARY KEY (exhibition_id, artwork_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id),
    FOREIGN KEY (artwork_id) REFERENCES artworks(artwork_id)
)
''')

# Visitors table
cursor.execute('''
CREATE TABLE IF NOT EXISTS visitors (
    visitor_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    membership_level TEXT
)
''')

# Visits table
cursor.execute('''
CREATE TABLE IF NOT EXISTS visits (
    visit_id INTEGER PRIMARY KEY,
    visitor_id INTEGER,
    exhibition_id INTEGER,
    visit_date DATE,
    ticket_price_paid DECIMAL(6, 2),
    duration_minutes INTEGER,
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id)
)
''')

conn.commit()
print("Schema created successfully")

Schema created successfully


In [3]:
# Insert sample data

# Artists
artists_data = [
    (1, 'Claude Monet', 1840, 'French', 'Impressionism'),
    (2, 'Vincent van Gogh', 1853, 'Dutch', 'Post-Impressionism'),
    (3, 'Pablo Picasso', 1881, 'Spanish', 'Cubism'),
    (4, 'Frida Kahlo', 1907, 'Mexican', 'Surrealism'),
    (5, 'Leonardo da Vinci', 1452, 'Italian', 'Renaissance'),
    (6, 'Rembrandt', 1606, 'Dutch', 'Baroque')
]
cursor.executemany('INSERT OR IGNORE INTO artists VALUES (?, ?, ?, ?, ?)', artists_data)

# Artworks
artworks_data = [
    (1, 'Water Lilies', 1, 1906, 'Oil on canvas', 40000000),
    (2, 'Impression, Sunrise', 1, 1872, 'Oil on canvas', 35000000),
    (3, 'The Starry Night', 2, 1889, 'Oil on canvas', 100000000),
    (4, 'Sunflowers', 2, 1888, 'Oil on canvas', 75000000),
    (5, 'Les Demoiselles d\'Avignon', 3, 1907, 'Oil on canvas', 90000000),
    (6, 'Guernica', 3, 1937, 'Oil on canvas', 200000000),
    (7, 'The Two Fridas', 4, 1939, 'Oil on canvas', 50000000),
    (8, 'Self-Portrait with Thorn Necklace', 4, 1940, 'Oil on canvas', 45000000),
    (9, 'Mona Lisa', 5, 1503, 'Oil on panel', 850000000),
    (10, 'The Last Supper', 5, 1498, 'Tempera', 600000000),
    (11, 'The Night Watch', 6, 1642, 'Oil on canvas', 75000000)
]
cursor.executemany('INSERT OR IGNORE INTO artworks VALUES (?, ?, ?, ?, ?, ?)', artworks_data)

# Exhibitions
exhibitions_data = [
    (1, 'Impressionist Masters', '2024-01-15', '2024-03-31', 'Main Gallery', 25.00),
    (2, 'Modern Art Revolution', '2024-02-01', '2024-04-30', 'East Wing', 30.00),
    (3, 'Renaissance Wonders', '2024-03-15', '2024-06-15', 'Grand Hall', 35.00),
    (4, 'Dutch Golden Age', '2024-04-01', '2024-05-31', 'North Gallery', 20.00)
]
cursor.executemany('INSERT OR IGNORE INTO exhibitions VALUES (?, ?, ?, ?, ?, ?)', exhibitions_data)

# Exhibition-Artworks relationships
exhibition_artworks_data = [
    (1, 1), (1, 2), (1, 3), (1, 4),  # Impressionist exhibition
    (2, 5), (2, 6), (2, 7), (2, 8),  # Modern Art exhibition
    (3, 9), (3, 10),  # Renaissance exhibition
    (4, 11)  # Dutch exhibition
]
cursor.executemany('INSERT OR IGNORE INTO exhibition_artworks VALUES (?, ?)', exhibition_artworks_data)

# Visitors
visitors_data = [
    (1, 'Sarah', 'Johnson', 'sarah.j@email.com', 'Premium'),
    (2, 'Michael', 'Chen', 'mchen@email.com', 'Standard'),
    (3, 'Emily', 'Rodriguez', 'emily.r@email.com', 'Premium'),
    (4, 'David', 'Kim', 'dkim@email.com', 'Standard'),
    (5, 'Jessica', 'Patel', 'jpatel@email.com', 'Basic'),
    (6, 'Robert', 'Taylor', 'rtaylor@email.com', 'Premium'),
    (7, 'Maria', 'Garcia', 'mgarcia@email.com', 'Standard'),
    (8, 'James', 'Wilson', 'jwilson@email.com', 'Basic')
]
cursor.executemany('INSERT OR IGNORE INTO visitors VALUES (?, ?, ?, ?, ?)', visitors_data)

# Visits
visits_data = [
    (1, 1, 1, '2024-01-20', 25.00, 120),
    (2, 1, 2, '2024-02-15', 30.00, 90),
    (3, 2, 1, '2024-01-25', 25.00, 75),
    (4, 3, 1, '2024-02-01', 25.00, 110),
    (5, 3, 3, '2024-03-20', 35.00, 150),
    (6, 4, 2, '2024-02-10', 30.00, 60),
    (7, 5, 1, '2024-01-30', 25.00, 45),
    (8, 6, 3, '2024-03-25', 35.00, 180),
    (9, 7, 2, '2024-02-20', 30.00, 85),
    (10, 8, 1, '2024-02-05', 25.00, 50),
    (11, 2, 4, '2024-04-15', 20.00, 70),
    (12, 4, 4, '2024-04-20', 20.00, 65)
]
cursor.executemany('INSERT OR IGNORE INTO visits VALUES (?, ?, ?, ?, ?, ?)', visits_data)

conn.commit()
print("Sample data inserted successfully")
print(f"\nArtists: {len(artists_data)}")
print(f"Artworks: {len(artworks_data)}")
print(f"Exhibitions: {len(exhibitions_data)}")
print(f"Visitors: {len(visitors_data)}")
print(f"Visits: {len(visits_data)}")

Sample data inserted successfully

Artists: 6
Artworks: 11
Exhibitions: 4
Visitors: 8
Visits: 12


---
## Problem 1: Data Exploration (10 minutes)

**Scenario**: The museum director wants to understand the current state of the database.

### Task 1.1: Write a query to show all exhibitions with their date ranges and ticket prices

In [6]:
# YOUR CODE HERE
query_1_1 = """
SELECT
    exhibition_name,
    start_date,
    end_date,
    ticket_price
FROM exhibitions
ORDER BY start_date;
"""

#Test your query
df_exhibitions = pd.read_sql_query(query_1_1, conn)
print(df_exhibitions)

         exhibition_name  start_date    end_date  ticket_price
0  Impressionist Masters  2024-01-15  2024-03-31            25
1  Modern Art Revolution  2024-02-01  2024-04-30            30
2    Renaissance Wonders  2024-03-15  2024-06-15            35
3       Dutch Golden Age  2024-04-01  2024-05-31            20


### Task 1.2: Find all artworks by Impressionist or Post-Impressionist artists

Include the artist name, artwork title, and creation year.

In [7]:
# YOUR CODE HERE
query_1_2 = """
SELECT
    ar.artist_name,
    aw.title AS artwork_title,
    aw.creation_year
FROM artworks aw
JOIN artists ar ON aw.artist_id = ar.artist_id
WHERE ar.movement IN ('Impressionism', 'Post-Impressionism')
ORDER BY aw.creation_year;
"""

# Test your query
df_impressionist = pd.read_sql_query(query_1_2, conn)
print(df_impressionist)

        artist_name        artwork_title  creation_year
0      Claude Monet  Impression, Sunrise           1872
1  Vincent van Gogh           Sunflowers           1888
2  Vincent van Gogh     The Starry Night           1889
3      Claude Monet         Water Lilies           1906


---
## Problem 2: Revenue Analysis (15 minutes)

**Scenario**: The CFO wants to analyze exhibition revenue and visitor patterns.

### Task 2.1: Calculate total revenue and visitor count for each exhibition

Your result should include:
- Exhibition name
- Total number of visits
- Total revenue
- Average visit duration

Order by total revenue (highest first).

In [8]:
# YOUR CODE HERE
query_2_1 = """
SELECT
    e.exhibition_name,
    COUNT(v.visit_id) AS total_visits,
    SUM(v.ticket_price_paid) AS total_revenue,
    ROUND(AVG(v.duration_minutes), 2) AS avg_visit_duration
FROM exhibitions e
LEFT JOIN visits v ON e.exhibition_id = v.exhibition_id
GROUP BY e.exhibition_id
ORDER BY total_revenue DESC;
"""

# Test your query
df_revenue = pd.read_sql_query(query_2_1, conn)
print(df_revenue)

         exhibition_name  total_visits  total_revenue  avg_visit_duration
0  Impressionist Masters             5            125               80.00
1  Modern Art Revolution             3             90               78.33
2    Renaissance Wonders             2             70              165.00
3       Dutch Golden Age             2             40               67.50


### Task 2.2: Identify high-value visitors

Create a query to find visitors who have:
- Made more than 2 visits
- Spent more than $50 total

Include their name, membership level, total visits, and total spent.

In [10]:
# YOUR CODE HERE
query_2_2 = """
SELECT
    vis.first_name || ' ' || vis.last_name AS visitor_name,
    vis.membership_level,
    COUNT(v.visit_id) AS total_visits,
    SUM(v.ticket_price_paid) AS total_spent
FROM visitors vis
JOIN visits v ON vis.visitor_id = v.visitor_id
GROUP BY vis.visitor_id
HAVING COUNT(v.visit_id) > 2
   AND SUM(v.ticket_price_paid) > 50
ORDER BY total_spent DESC;
"""

# Test your query
df_high_value = pd.read_sql_query(query_2_2, conn)
print(df_high_value)

Empty DataFrame
Columns: [visitor_name, membership_level, total_visits, total_spent]
Index: []


---
## Problem 3: Complex Analysis (15 minutes)

**Scenario**: The museum wants to optimize their exhibition strategy.

### Task 3.1: Calculate the value-to-visitors ratio

For each exhibition, calculate:
- Total value of artworks displayed (sum of artwork values)
- Number of unique visitors
- Value per visitor (total artwork value / number of visitors)

This helps understand which exhibitions attract visitors relative to their collection value.

In [11]:
# YOUR CODE HERE
query_3_1 = """
SELECT
    e.exhibition_name,
    SUM(aw.value_usd) AS total_artwork_value,
    COUNT(DISTINCT v.visitor_id) AS unique_visitors,
    ROUND(
        SUM(aw.value_usd) / COUNT(DISTINCT v.visitor_id), 2
    ) AS value_per_visitor
FROM exhibitions e
JOIN exhibition_artworks ea ON e.exhibition_id = ea.exhibition_id
JOIN artworks aw ON ea.artwork_id = aw.artwork_id
LEFT JOIN visits v ON e.exhibition_id = v.exhibition_id
GROUP BY e.exhibition_id;
"""

# Test your query
df_value_analysis = pd.read_sql_query(query_3_1, conn)
print(df_value_analysis)

         exhibition_name  total_artwork_value  unique_visitors  \
0  Impressionist Masters           1250000000                5   
1  Modern Art Revolution           1155000000                3   
2    Renaissance Wonders           2900000000                2   
3       Dutch Golden Age            150000000                2   

   value_per_visitor  
0       2.500000e+08  
1       3.850000e+08  
2       1.450000e+09  
3       7.500000e+07  


### Task 3.2: Find the most popular artist

Determine which artist's works have been viewed the most (total visits to exhibitions featuring their work).

Show:
- Artist name
- Number of their artworks in exhibitions
- Total visits to exhibitions featuring their work
- Average visit duration for exhibitions featuring their work

In [12]:
# YOUR CODE HERE
query_3_2 = """
SELECT
    ar.artist_name,
    COUNT(DISTINCT aw.artwork_id) AS artwork_count,
    COUNT(v.visit_id) AS total_visits,
    ROUND(AVG(v.duration_minutes), 2) AS avg_visit_duration
FROM artists ar
JOIN artworks aw ON ar.artist_id = aw.artist_id
JOIN exhibition_artworks ea ON aw.artwork_id = ea.artwork_id
JOIN exhibitions e ON ea.exhibition_id = e.exhibition_id
JOIN visits v ON e.exhibition_id = v.exhibition_id
GROUP BY ar.artist_id
ORDER BY total_visits DESC;
"""

# Test your query
df_popular_artists = pd.read_sql_query(query_3_2, conn)
print(df_popular_artists)

         artist_name  artwork_count  total_visits  avg_visit_duration
0   Vincent van Gogh              2            10               80.00
1       Claude Monet              2            10               80.00
2        Frida Kahlo              2             6               78.33
3      Pablo Picasso              2             6               78.33
4  Leonardo da Vinci              2             4              165.00
5          Rembrandt              1             2               67.50


---
## Problem 4: Python Data Analysis (15 minutes)

**Scenario**: The marketing team wants insights that require more complex analysis than SQL can easily provide.

### Task 4.1: Visitor Segmentation Analysis

Using pandas, analyze visitor behavior by membership level:
1. Load all visits with visitor information
2. Calculate for each membership level:
   - Average spending per visit
   - Average visit duration
   - Number of unique visitors
   - Total revenue contribution
3. Create a summary visualization or table

In [19]:
# YOUR CODE HERE
# Step 1: Load the data
query = """
SELECT
    v.visit_id,
    v.visitor_id,
    vis.membership_level,
    v.ticket_price_paid,
    v.duration_minutes
FROM visits v
JOIN visitors vis ON v.visitor_id = vis.visitor_id
"""

df_visits = pd.read_sql_query(query, conn)

# Step 2: Perform your analysis
summary_df = (
    df_visits
    .groupby("membership_level")
    .agg(
        avg_spending_per_visit=("ticket_price_paid", "mean"),
        avg_visit_duration=("duration_minutes", "mean"),
        unique_visitors=("visitor_id", "nunique"),
        total_revenue=("ticket_price_paid", "sum")
    )
    .reset_index()
)


# Step 3: Display results
summary_df


,membership_level,avg_spending_per_visit,avg_visit_duration,unique_visitors,total_revenue
0,Basic,25.0,47.5,2,50
1,Premium,30.0,130.0,3,150
2,Standard,25.0,71.0,3,125


### Task 4.2: Time-based Trend Analysis

Create a time series analysis showing:
1. Daily visitor count
2. 7-day rolling average of visitors
3. Identify the busiest day of the week

**Hint**: You may need to parse dates and use pandas datetime functionality.

In [20]:
# YOUR CODE HERE
# Load visit dates
query = """
SELECT
    visit_date,
    COUNT(*) as visit_count
FROM visits
GROUP BY visit_date
ORDER BY visit_date
"""

df_daily = pd.read_sql_query(query, conn)

# Convert to datetime and perform analysis
df_daily["visit_date"] = pd.to_datetime(df_daily["visit_date"])
df_daily.set_index("visit_date", inplace=True)

df_daily["rolling_7_day_avg"] = df_daily["visit_count"].rolling(7).mean()

# Busiest weekday
df_daily["weekday"] = df_daily.index.day_name()
busiest_day = (
    df_daily.groupby("weekday")["visit_count"]
    .sum()
    .idxmax()
)

df_daily, busiest_day


(            visit_count  rolling_7_day_avg    weekday
 visit_date                                           
 2024-01-20            1                NaN   Saturday
 2024-01-25            1                NaN   Thursday
 2024-01-30            1                NaN    Tuesday
 2024-02-01            1                NaN   Thursday
 2024-02-05            1                NaN     Monday
 2024-02-10            1                NaN   Saturday
 2024-02-15            1                1.0   Thursday
 2024-02-20            1                1.0    Tuesday
 2024-03-20            1                1.0  Wednesday
 2024-03-25            1                1.0     Monday
 2024-04-15            1                1.0     Monday
 2024-04-20            1                1.0   Saturday,
 'Monday')

---
## Problem 5: Database Design Challenge (15 minutes)

**Scenario**: The museum wants to expand their system to include:
- Audio guides (which visitors can rent)
- Feedback/ratings for exhibitions
- Gift shop purchases

### Task 5.1: Design the Schema

Create an ERD (Entity-Relationship Diagram) description or SQL schema for:

1. **Audio Guides**: Track which visitors rent audio guides, rental fee, return status
2. **Exhibition Ratings**: Allow visitors to rate exhibitions (1-5 stars) with optional comments
3. **Gift Shop**: Track items, prices, and purchases by visitors

Write the CREATE TABLE statements with appropriate:
- Primary keys
- Foreign keys
- Data types
- Constraints

In [21]:
# YOUR CODE HERE
# Write CREATE TABLE statements for the three new features

# Audio Guides table
audio_guides_schema = """
CREATE TABLE audio_guide_rentals (
    rental_id INTEGER PRIMARY KEY,
    visitor_id INTEGER NOT NULL,
    exhibition_id INTEGER NOT NULL,
    rental_date DATE NOT NULL,
    rental_fee DECIMAL(6,2),
    returned BOOLEAN DEFAULT 0,
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id)
);
"""

# Exhibition Ratings table
ratings_schema = """
CREATE TABLE exhibition_ratings (
    rating_id INTEGER PRIMARY KEY,
    visitor_id INTEGER NOT NULL,
    exhibition_id INTEGER NOT NULL,
    rating INTEGER CHECK (rating BETWEEN 1 AND 5),
    comment TEXT,
    rating_date DATE,
    UNIQUE(visitor_id, exhibition_id),
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id)
);
"""

# Gift Shop tables (you might need multiple tables)
gift_shop_schema = """
CREATE TABLE gift_shop_items (
    item_id INTEGER PRIMARY KEY,
    item_name TEXT NOT NULL,
    price DECIMAL(6,2)
);

CREATE TABLE gift_shop_purchases (
    purchase_id INTEGER PRIMARY KEY,
    visitor_id INTEGER,
    item_id INTEGER,
    purchase_date DATE,
    quantity INTEGER DEFAULT 1,
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (item_id) REFERENCES gift_shop_items(item_id)
);
"""

# Print your schemas
print("Audio Guides Schema:")
print(audio_guides_schema)
print("\nRatings Schema:")
print(ratings_schema)
print("\nGift Shop Schema:")
print(gift_shop_schema)

Audio Guides Schema:

CREATE TABLE audio_guide_rentals (
    rental_id INTEGER PRIMARY KEY,
    visitor_id INTEGER NOT NULL,
    exhibition_id INTEGER NOT NULL,
    rental_date DATE NOT NULL,
    rental_fee DECIMAL(6,2),
    returned BOOLEAN DEFAULT 0,
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id)
);


Ratings Schema:

CREATE TABLE exhibition_ratings (
    rating_id INTEGER PRIMARY KEY,
    visitor_id INTEGER NOT NULL,
    exhibition_id INTEGER NOT NULL,
    rating INTEGER CHECK (rating BETWEEN 1 AND 5),
    comment TEXT,
    rating_date DATE,
    UNIQUE(visitor_id, exhibition_id),
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id)
);


Gift Shop Schema:

CREATE TABLE gift_shop_items (
    item_id INTEGER PRIMARY KEY,
    item_name TEXT NOT NULL,
    price DECIMAL(6,2)
);

CREATE TABLE gift_shop_purchases (
    purchase_i

### Task 5.2: Explain Your Design Decisions

In the cell below, explain:
1. Why you chose specific data types
2. How you handled relationships between tables
3. Any assumptions you made
4. How your design supports common queries

**YOUR EXPLANATION HERE:**

#### 1. Choice of Data Types
Integer primary keys were used for all tables to ensure efficient indexing and fast joins.  
Monetary values such as ticket prices and rental fees use `DECIMAL` to avoid floating-point precision issues.  
Date fields (`rental_date`, `rating_date`, `purchase_date`) use the `DATE` data type to support time-based analysis and reporting.  
Boolean values (such as audio guide return status) are stored as integers for compatibility with SQLite.

#### 2. Handling Relationships Between Tables
Relationships are enforced using foreign keys.  
Visitor-related activities (audio guide rentals, exhibition ratings, and gift shop purchases) reference the `visitors` table.  
Exhibition-related data references the `exhibitions` table.  
This design maintains referential integrity and allows consistent tracking of visitor interactions across museum services.

#### 3. Assumptions Made
- A visitor can rent multiple audio guides across different exhibitions.
- Each visitor can rate an exhibition only once, enforced using a unique constraint.
- Gift shop purchases are tracked at the item level with quantity support.
- Prices are stored at the time of transaction to preserve historical accuracy.

#### 4. Support for Common Queries
This schema supports common analytical and operational queries, including:
- Revenue analysis by exhibition, visitor, or service type
- Average exhibition ratings and visitor feedback trends
- Identification of popular gift shop items
- Analysis of add-on service usage (audio guides) by membership level

---
## Bonus Challenge: Advanced Analytics (Optional)

**Scenario**: The museum wants predictive insights.

### Bonus Task: Revenue Forecasting

Based on the current data:
1. Calculate the average revenue per exhibition per day
2. Identify seasonal or temporal patterns
3. Predict potential revenue for a new 60-day exhibition
4. Recommend optimal pricing based on historical data

Use both SQL and Python for this analysis.

In [ ]:
# YOUR CODE HERE
# This is an open-ended problem - show your analytical thinking!


---
## Problem 6: Data Quality & Cleaning (10 minutes)

**Scenario**: You discover some data quality issues in the database.

### Task 6.1: Identify Data Quality Issues

Write queries to find:
1. Any visits where ticket_price_paid doesn't match the exhibition's ticket_price
2. Artworks without an associated artist
3. Exhibitions with no artwork assigned
4. Any duplicate visitor emails

In [24]:
# YOUR CODE HERE
# Write queries for each data quality check

# 1. Price mismatches: visits where ticket_price_paid does not match exhibition ticket_price
query_price_check = """
SELECT
    v.visit_id,
    v.visitor_id,
    e.exhibition_name,
    v.ticket_price_paid,
    e.ticket_price
FROM visits v
JOIN exhibitions e
    ON v.exhibition_id = e.exhibition_id
WHERE v.ticket_price_paid != e.ticket_price;
"""

# 2. Orphaned artworks: artworks without a valid artist
query_orphaned_artworks = """
SELECT
    artwork_id,
    title,
    artist_id
FROM artworks
WHERE artist_id IS NULL
   OR artist_id NOT IN (SELECT artist_id FROM artists);
"""

# 3. Empty exhibitions: exhibitions with no artwork assigned
query_empty_exhibitions = """
SELECT
    e.exhibition_id,
    e.exhibition_name
FROM exhibitions e
LEFT JOIN exhibition_artworks ea
    ON e.exhibition_id = ea.exhibition_id
WHERE ea.artwork_id IS NULL;
"""

# 4. Duplicate visitor emails: emails used by more than one visitor
query_duplicate_emails = """
SELECT
    email,
    COUNT(*) AS count
FROM visitors
GROUP BY email
HAVING COUNT(*) > 1;
"""

# Run your checks
print("Price Mismatches:")
print(pd.read_sql_query(query_price_check, conn))

print("\nOrphaned Artworks:")
print(pd.read_sql_query(query_orphaned_artworks, conn))

print("\nEmpty Exhibitions:")
print(pd.read_sql_query(query_empty_exhibitions, conn))

print("\nDuplicate Visitor Emails:")
print(pd.read_sql_query(query_duplicate_emails, conn))

Price Mismatches:
Empty DataFrame
Columns: [visit_id, visitor_id, exhibition_name, ticket_price_paid, ticket_price]
Index: []

Orphaned Artworks:
Empty DataFrame
Columns: [artwork_id, title, artist_id]
Index: []

Empty Exhibitions:
Empty DataFrame
Columns: [exhibition_id, exhibition_name]
Index: []

Duplicate Visitor Emails:
Empty DataFrame
Columns: [email, count]
Index: []


---
## Reflection Questions

After completing the exercises, consider these interview discussion points:

1. **Performance Optimization**:
   - Which queries might be slow with millions of records?
   - What indexes would you create?
   - How would you optimize the slowest query?

2. **Scalability**:
   - How would you modify this design for multiple museums?
   - What if visitor data grows to 10 million records?
   - How would you handle real-time analytics?

3. **Data Governance**:
   - What privacy concerns exist with visitor data?
   - How would you implement data retention policies?
   - What audit trails might be necessary?

4. **Business Intelligence**:
   - What KPIs would you track?
   - How would you build a dashboard?
   - What predictive models might be useful?

5. **Technical Decisions**:
   - Why use a relational database vs. NoSQL for this application?
   - When would you denormalize for performance?
   - How would you handle concurrent updates?

Be prepared to discuss these in your interview!

---
## Solutions Reference

After attempting all problems, you can check your approach against best practices:

### Key Learning Points:

1. **SQL Mastery**:
   - Comfortable with multi-table JOINs
   - Use of aggregate functions with GROUP BY
   - Subqueries and CTEs for complex logic
   - Window functions for ranking and analytics

2. **Data Modeling**:
   - Proper use of primary and foreign keys
   - Understanding of many-to-many relationships
   - Normalization principles
   - Constraint design

3. **Python/Pandas**:
   - Loading data from SQL
   - GroupBy operations
   - DateTime handling
   - Data cleaning techniques

4. **Analytical Thinking**:
   - Breaking down complex problems
   - Choosing appropriate tools (SQL vs. Python)
   - Performance considerations
   - Business context awareness

---

**Good luck with your interview!**

In [ ]:
# Clean up
conn.close()
print("Database connection closed")